In [1]:
import os
import time
import warnings

import cv2
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import accuracy_score

In [5]:
def preparar_dataset_faces(dimensoes_resize, arquivo_saida, usar_pca=False, n_componentes=None):
    """
    Carrega imagens de faces, as pré-processa e salva o dataset em um arquivo.

    A função busca por imagens de faces em um formato de nome específico no diretório
    atual, redimensiona cada imagem, as vetoriza, opcionalmente aplica PCA para
    redução de dimensionalidade, e as salva com seus rótulos em um arquivo de texto.

    Args:
        dimensoes_resize (tuple): Tupla (altura, largura) para o redimensionamento. Ex: (60, 60).
        arquivo_saida (str): Nome do arquivo de texto onde o dataset será salvo.
        usar_pca (bool, optional): Se True, aplica PCA nos dados. Padrão é False.
        n_componentes (int, optional): Número de componentes principais a serem mantidos.
                                       Obrigatório se usar_pca for True.

    Returns:
        bool: True se o dataset foi criado com sucesso, False caso contrário.
    """
    print("--- Iniciando a preparação do dataset de faces ---")

    # --- Validação de Parâmetros ---
    if usar_pca and (n_componentes is None or n_componentes <= 0):
        print("\nErro: Se 'usar_pca' é True, 'n_componentes' deve ser um número inteiro positivo.")
        return False

    # --- Configuração dos nomes de arquivo ---
    part1 = 'subject0'
    part2 = 'subject'
    part3 = [
        '.centerlight', '.glasses', '.happy', '.leftlight', '.noglasses', 
        '.normal', '.rightlight', '.sad', '.sleepy', '.surprised', '.wink'
    ]
    Nind = 15  # Número de indivíduos (classes)

    X_list = []
    Y_list = []

    # --- Loop de carregamento e processamento ---
    print(f"Redimensionando imagens para {dimensoes_resize[0]}x{dimensoes_resize[1]} pixels...")
    for i in range(1, Nind + 1):
        for expression in part3:
            if i < 10:
                nome = f"{part1}{i}{expression}"
            else:
                nome = f"{part2}{i}{expression}"
            
            if not os.path.exists(nome):
                continue

            img = cv2.imread(nome, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                continue

            ar = cv2.resize(img, (dimensoes_resize[1], dimensoes_resize[0]))
            a_mat = ar.astype(np.float64) / 255.0
            a_vec = a_mat.flatten(order='F')
            rot = i

            X_list.append(a_vec)
            Y_list.append(rot)
    
    # --- Verificação e montagem do dataset ---
    if not X_list:
        print("\nErro: Nenhuma imagem foi encontrada. O dataset não foi gerado.")
        return False

    X = np.array(X_list)
    Y = np.array(Y_list).reshape(-1, 1)

    # --- Etapa opcional de PCA ---
    if usar_pca:
        # Verifica se o número de componentes não é maior que o número de amostras
        if n_componentes > X.shape[0]:
            print(f"\nAviso: O número de componentes ({n_componentes}) é maior que o número de amostras ({X.shape[0]}).")
            print(f"Ajustando o número de componentes para {X.shape[0]}.")
            n_componentes = X.shape[0]

        print(f"\nAplicando PCA para reduzir a dimensionalidade para {n_componentes} componentes...")
        pca = PCA(n_components=n_componentes)
        X = pca.fit_transform(X) # fit_transform espera amostras como linhas
        print(f"Dimensionalidade reduzida. Novo formato das features: {X.shape}")

    # Combina features (X) e rótulo (Y) em uma única matriz Z
    Z = np.hstack((X, Y))

    # --- Salvamento do arquivo ---
    try:
        np.savetxt(arquivo_saida, Z, fmt='%.8f')
        print(f"\nProcessamento completo.")
        print(f"Dataset com {Z.shape[0]} amostras e {Z.shape[1]} colunas salvo em '{arquivo_saida}'.")
        return True
    except Exception as e:
        print(f"\nErro ao salvar o arquivo: {e}")
        return False

def executar_analise_modelos(nome_arquivo):
    """
    Executa uma análise comparativa de modelos de regressão para uma tarefa de classificação.

    A função carrega os dados de um arquivo, executa uma validação cruzada de 5 folds
    para cada modelo, calcula estatísticas de acurácia e tempo, e imprime uma
    tabela de resultados formatada no console.

    Args:
        nome_arquivo (str): O caminho para o arquivo de dados (ex: 'recfaces.dat').

    Returns:
        pandas.DataFrame or None: Um DataFrame contendo os resultados compilados da análise,
                                  ou None se o arquivo de dados não for encontrado.
    """
    # --- Configuração Inicial ---
    # Ignorar avisos para manter a saída limpa
    from sklearn.exceptions import ConvergenceWarning
    warnings.filterwarnings('ignore', category=ConvergenceWarning)

    # --- Carregar os Dados ---
    try:
        print(f"Carregando o arquivo '{nome_arquivo}'...")
        data = np.loadtxt(nome_arquivo)
        X = data[:, :-1]
        y = data[:, -1]
        print(f"Dados carregados: {X.shape[0]} amostras, {X.shape[1]} features.")
        print("-" * 50)
    except FileNotFoundError:
        print(f"Erro: O arquivo '{nome_arquivo}' não foi encontrado.")
        return None

    # --- Definir os Modelos de Regressão ---
    regressors = {
        "MQ": LinearRegression(),
        "PL": MLPRegressor(hidden_layer_sizes=(), activation='relu',
                                                  max_iter=100000, random_state=2,solver='sgd'),
        "MLP-1H": MLPRegressor(hidden_layer_sizes=(10,), activation='relu',
                                              max_iter=100000, random_state=2,solver='sgd'),
        "MLP-2H": MLPRegressor(hidden_layer_sizes=(10, 10), activation='relu',
                                                 max_iter=10000, random_state=2,solver='sgd')
    }

    # --- Preparação para Coletar Resultados ---
    results_list = []

    # --- Loop de Execução e Avaliação com Validação Cruzada ---
    for name, model in regressors.items():
        print(f"Avaliando o modelo: {name}...")

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('regressor', model)
        ])

        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        fold_accuracies = []
        fold_iterations = []
        fold_times = []
        
        for train_index, test_index in kf.split(X, y):
            start_fold_time = time.time()
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            pipeline.fit(X_train, y_train)
            y_pred_continuous = pipeline.predict(X_test)
            y_pred_class = np.round(y_pred_continuous)
            y_pred_class = np.clip(y_pred_class, 1, 15)
            
            end_fold_time = time.time()
            
            fold_accuracies.append(accuracy_score(y_test, y_pred_class))
            fold_times.append(end_fold_time - start_fold_time)

            if hasattr(pipeline.named_steps['regressor'], 'n_iter_'):
                fold_iterations.append(pipeline.named_steps['regressor'].n_iter_)

        result_dict = {
            'Classificador': name,
            'Acurácia Média': np.mean(fold_accuracies),
            'Acurácia Mín': np.min(fold_accuracies),
            'Acurácia Máx': np.max(fold_accuracies),
            'Desvio Padrão': np.std(fold_accuracies),
            'Tempo Médio / Fold (s)': np.mean(fold_times),
            'Iterações (Média)': np.mean(fold_iterations) if fold_iterations else np.nan
        }
        
        results_list.append(result_dict)

    # --- Apresentação Final dos Resultados ---
    print("\n" + "="*70)
    print("--- Tabela de Resultados (Validação Cruzada com 5 Folds) ---")
    print("="*70)

    results_df = pd.DataFrame(results_list)
    pd.options.display.float_format = '{:,.4f}'.format
    results_df = results_df.rename(columns={
        'Acurácia Média': 'Média',
        'Acurácia Mín': 'Mínimo',
        'Acurácia Máx': 'Máximo'
    })

    # print(results_df[[
    #     'Classificador', 'Média', 'Mínimo', 'Máximo', 'Desvio Padrão', 
    #     'Tempo Médio / Fold (s)', 'Iterações (Média)'
    # ]].to_string(index=False))
    
    # Retorna o DataFrame para uso posterior, se necessário
    return results_df

In [6]:
preparar_dataset_faces(dimensoes_resize=(60, 60), arquivo_saida='recfaces_60x60_sem_pca.dat', usar_pca=False)
executar_analise_modelos('recfaces_60x60_sem_pca.dat')

--- Iniciando a preparação do dataset de faces ---
Redimensionando imagens para 60x60 pixels...

Processamento completo.
Dataset com 165 amostras e 3601 colunas salvo em 'recfaces_60x60_sem_pca.dat'.
Carregando o arquivo 'recfaces_60x60_sem_pca.dat'...
Dados carregados: 165 amostras, 3600 features.
--------------------------------------------------
Avaliando o modelo: MQ...
Avaliando o modelo: PL...
Avaliando o modelo: MLP-1H...
Avaliando o modelo: MLP-2H...

--- Tabela de Resultados (Validação Cruzada com 5 Folds) ---


,Classificador,Média,Mínimo,Máximo,Desvio Padrão,Tempo Médio / Fold (s),Iterações (Média)
0,MQ,0.3333,0.2727,0.3636,0.0332,0.0220,NaN
1,PL,0.3576,0.3030,0.4242,0.0555,0.1692,423.6000
2,MLP-1H,0.4485,0.3030,0.6061,0.1124,0.1734,173.8000
3,MLP-2H,0.3818,0.3030,0.4848,0.0624,0.1379,167.8000


In [10]:
preparar_dataset_faces(dimensoes_resize=(60, 60), arquivo_saida='recfaces_60x60_com_pca.dat', usar_pca=True, n_componentes=20)
executar_analise_modelos('recfaces_60x60_com_pca.dat')

--- Iniciando a preparação do dataset de faces ---
Redimensionando imagens para 60x60 pixels...

Aplicando PCA para reduzir a dimensionalidade para 20 componentes...
Dimensionalidade reduzida. Novo formato das features: (165, 20)

Processamento completo.
Dataset com 165 amostras e 21 colunas salvo em 'recfaces_60x60_com_pca.dat'.
Carregando o arquivo 'recfaces_60x60_com_pca.dat'...
Dados carregados: 165 amostras, 20 features.
--------------------------------------------------
Avaliando o modelo: MQ...
Avaliando o modelo: PL...
Avaliando o modelo: MLP-1H...
Avaliando o modelo: MLP-2H...

--- Tabela de Resultados (Validação Cruzada com 5 Folds) ---


,Classificador,Média,Mínimo,Máximo,Desvio Padrão,Tempo Médio / Fold (s),Iterações (Média)
0,MQ,0.2242,0.0606,0.3333,0.0951,0.0008,NaN
1,PL,0.2121,0.0606,0.3030,0.0857,0.0263,440.2000
2,MLP-1H,0.4303,0.3333,0.4848,0.0555,0.2562,"1,890.8000"
3,MLP-2H,0.4303,0.4242,0.4545,0.0121,0.1813,"1,236.6000"
